# 6.01 N-gram



In [3]:
import nltk


# 이 함수는 문장을 입력받아 n-그램을 생성하는 함수임
def ngrams(sentence, n):
    # 문장을 공백을 기준으로 나누어 단어 리스트로 변환함
    words = sentence.split()
    # 주어진 n값에 따라 단어들을 묶어 n-그램 리스트를 생성함
    ngrams = zip(*[words[i:] for i in range(n)])
    return list(ngrams)

sentence = "안녕 오랜만이야! 만나서 반가워~"

# 각 n에 따라 n-그램을 생성해봄
unigram = ngrams(sentence, 1)  # 유니그램
bigram = ngrams(sentence, 2)   # 바이그램
trigram = ngrams(sentence, 3)  # 트라이그램

# 생성된 n-그램을 출력함
print(unigram)
print(bigram)
print(trigram)

# NLTK 라이브러리를 활용하여 n-그램을 생성해봄
unigram = nltk.ngrams(sentence.split(), 1)
bigram = nltk.ngrams(sentence.split(), 2)
trigram = nltk.ngrams(sentence.split(), 3)

# NLTK로 생성한 n-그램을 출력함
print(list(unigram))
print(list(bigram))
print(list(trigram))

[('안녕',), ('오랜만이야!',), ('만나서',), ('반가워~',)]
[('안녕', '오랜만이야!'), ('오랜만이야!', '만나서'), ('만나서', '반가워~')]
[('안녕', '오랜만이야!', '만나서'), ('오랜만이야!', '만나서', '반가워~')]
[('안녕',), ('오랜만이야!',), ('만나서',), ('반가워~',)]
[('안녕', '오랜만이야!'), ('오랜만이야!', '만나서'), ('만나서', '반가워~')]
[('안녕', '오랜만이야!', '만나서'), ('오랜만이야!', '만나서', '반가워~')]


## 6.02 TF-IDF 계산

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer


corpus = [
    "I like to play football",
    "My favorite sport is football",
    "I don’t like to play basketball",
    "My favorite sport is basketball"
]

# TF-IDF 벡터라이저를 초기화
tfidf_vectorizer = TfidfVectorizer()

# 코퍼스를 바탕으로 단어 사전을 학습함
tfidf_vectorizer.fit(corpus)

# 학습된 벡터라이저로 코퍼스를 변환하여 TF-IDF 행렬을 만듦
tfidf_matrix = tfidf_vectorizer.transform(corpus)

# TF-IDF 행렬을 배열로 변환하여 출력함
print(tfidf_matrix.toarray())

# TF-IDF 벡터라이저에 저장된 단어 사전 출력함
print(tfidf_vectorizer.vocabulary_)

[[0.         0.         0.         0.5        0.         0.5
  0.         0.5        0.         0.5       ]
 [0.         0.         0.4472136  0.4472136  0.4472136  0.
  0.4472136  0.         0.4472136  0.        ]
 [0.4222466  0.53556627 0.         0.         0.         0.4222466
  0.         0.4222466  0.         0.4222466 ]
 [0.4472136  0.         0.4472136  0.         0.4472136  0.
  0.4472136  0.         0.4472136  0.        ]]
{'like': 5, 'to': 9, 'play': 7, 'football': 3, 'my': 6, 'favorite': 2, 'sport': 8, 'is': 4, 'don': 1, 'basketball': 0}


## 6.03~6.12 Skip-gram 모델 실습

In [3]:
# 필수 라이브러리 설치
!pip install torch pandas Korpora konlpy numpy

In [2]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt
from collections import Counter
import numpy as np
from numpy.linalg import norm

### Word2Vec 모델 클래스 정의

In [3]:
# VanillaSkipgram 클래스는 단순한 스킵그램(word2vec) 모델 구조를 정의함
class VanillaSkipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        # 단어 임베딩을 위한 층을 정의함
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim
        )
        # 임베딩 벡터를 통해 단어 분류를 위한 출력층을 정의함
        self.linear = nn.Linear(
            in_features=embedding_dim,
            out_features=vocab_size
        )

    # 순전파 계산을 정의함
    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        output = self.linear(embeddings)
        return output

### 데이터 처리 및 토큰화


In [4]:
# 데이터 로드 및 토큰화
corpus = Korpora.load("nsmc")
corpus = pd.DataFrame(corpus.test)
tokenizer = Okt()
tokens = [tokenizer.morphs(review) for review in corpus.text]


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /root/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /root/Korpora/nsmc/ra

### 어휘 사전 구축


In [4]:
# 단어 사전을 구축하는 함수임
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    # 각 문서의 토큰을 세어서 카운트를 누적함
    for tokens in corpus:
        counter.update(tokens)
    # 특수 토큰을 추가하고 상위 n_vocab 개의 단어만 사전에 추가함
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

# 사전을 구축하고 각 단어와 인덱스를 매핑함
vocab = build_vocab(corpus=tokens, n_vocab=5000, special_tokens=["<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

### 단어 쌍 및 인덱스 생성


In [5]:
# 중심 단어와 주변 단어 쌍을 만드는 함수
def get_word_pairs(tokens, window_size):
    pairs = []
    for sentence in tokens:
        sentence_length = len(sentence)
        for idx, center_word in enumerate(sentence):
            window_start = max(0, idx - window_size)
            window_end = min(sentence_length, idx + window_size + 1)
            context_words = sentence[window_start:idx] + sentence[idx+1:window_end]
            for context_word in context_words:
                pairs.append([center_word, context_word])
    return pairs

word_pairs = get_word_pairs(tokens, window_size=2)

# 단어 쌍을 인덱스 쌍으로 변환하는 함수
def get_index_pairs(word_pairs, token_to_id):
    pairs = []
    unk_index = token_to_id["<unk>"]
    for word_pair in word_pairs:
        center_word, context_word = word_pair
        center_index = token_to_id.get(center_word, unk_index)
        context_index = token_to_id.get(context_word, unk_index)
        pairs.append([center_index, context_index])
    return pairs

index_pairs = get_index_pairs(word_pairs, token_to_id)

### 데이터셋 및 모델 학습


In [ ]:
# 텐서 데이터셋 생성 및 DataLoader 설정
index_pairs = torch.tensor(index_pairs)
center_indexs = index_pairs[:, 0]
context_indexs = index_pairs[:, 1]

# 배치 학습을 위해 텐서 데이터셋과 DataLoader로 구성함
dataset = TensorDataset(center_indexs, context_indexs)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 모델 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
word2vec = VanillaSkipgram(vocab_size=len(token_to_id), embedding_dim=128).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(word2vec.parameters(), lr=0.1)

# 학습 루프
for epoch in range(10):
    cost = 0.0
    for input_ids, target_ids in dataloader:
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)

        # 모델에 입력을 전달하여 출력 로그잇을 얻음
        logits = word2vec(input_ids)
        # 손실을 계산함
        loss = criterion(logits, target_ids)

        # 역전파 및 가중치 갱신을 수행함
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    # 에포크별 평균 손실을 출력함
    cost = cost / len(dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

### 임베딩 추출 및 유사도 계산


In [ ]:
# 임베딩 매트릭스 및 유사도 계산
token_to_embedding = dict()
embedding_matrix = word2vec.embedding.weight.detach().cpu().numpy()

# 각 단어의 임베딩을 사전에 저장함
for word, embedding in zip(vocab, embedding_matrix):
    token_to_embedding[word] = embedding

index = 30
token = vocab[30]
token_embedding = token_to_embedding[token]

# 코사인 유사도를 계산하는 함수임
def cosine_similarity(a, b):
    cosine = np.dot(b, a) / (norm(b, axis=1) * norm(a))
    return cosine

# 유사도가 가장 높은 단어 인덱스를 찾는 함수임
def top_n_index(cosine_matrix, n):
    closest_indexes = cosine_matrix.argsort()[::-1]
    top_n = closest_indexes[1 : n + 1]
    return top_n

# 코사인 유사도를 계산하고 상위 5개 단어를 찾음
cosine_matrix = cosine_similarity(token_embedding, embedding_matrix)
top_n = top_n_index(cosine_matrix, n=5)

# 가장 유사한 단어와 유사도를 출력함
print(f"{token}와 가장 유사한 5 개 단어")
for index in top_n:
    print(f"{id_to_token[index]} - 유사도 : {cosine_matrix[index]:.4f}")

## 6.13~6.14 Word2Vec 모델 실습

In [ ]:
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt


corpus = Korpora.load("nsmc")
corpus = pd.DataFrame(corpus.test)

In [ ]:
tokenizer = Okt()
tokens = [tokenizer.morphs(review) for review in corpus.text]

In [ ]:
from gensim.models import Word2Vec


word2vec = Word2Vec(
    sentences=tokens,
    vector_size=128,
    window=5,
    min_count=1,
    sg=1,
    epochs=3,
    max_final_vocab=10000
)

In [ ]:
word2vec.save("../models/word2vec.model")
word2vec = Word2Vec.load("../models/word2vec.model")

In [ ]:
word = "연기"
print(word2vec.wv[word])
print(word2vec.wv.most_similar(word, topn=5))
print(word2vec.wv.similarity(w1=word, w2="연기력"))

## 6.15~6.17 fastText OOV 모델 실습

In [ ]:
from Korpora import Korpora


corpus = Korpora.load("kornli")
corpus_texts = corpus.get_all_texts() + corpus.get_all_pairs()
tokens = [sentence.split() for sentence in corpus_texts]

print(tokens[:3])

In [ ]:
from gensim.models import FastText


fastText = FastText(
    sentences=tokens,
    vector_size=128,
    window=5,
    min_count=5,
    sg=1,
    epochs=3,
    min_n=2,
    max_n=6
)

# fastText.save("../models/fastText.model")
# fastText = FastText.load("../models/fastText.model")

In [ ]:
oov_token = "사랑해요"
oov_vector = fastText.wv[oov_token]

print(oov_token in fastText.wv.index_to_key)
print(fastText.wv.most_similar(oov_vector, topn=5))

## 6.18 양방향 다층 신경망

In [ ]:
import torch
from torch import nn

# 모델의 입력 크기와 출력 크기, RNN 층의 개수 및 양방향 설정을 정의함
input_size = 128  # 입력 벡터의 차원 크기
ouput_size = 256  # RNN의 은닉 상태(hidden state) 크기
num_layers = 3    # RNN 계층 수
bidirectional = True  # 양방향 RNN 설정 여부

# RNN 모델을 초기화함
model = nn.RNN(
    input_size=input_size,  # 입력 벡터의 차원 크기를 설정
    hidden_size=ouput_size,  # 은닉 상태의 차원 크기를 설정
    num_layers=num_layers,   # RNN 계층 수를 설정
    nonlinearity="tanh",     # 활성화 함수로 tanh 사용
    batch_first=True,        # 입력의 첫 번째 차원이 배치(batch) 크기임을 설정
    bidirectional=bidirectional,  # 양방향 RNN 설정
)

# 입력 데이터 설정
batch_size = 4        # 배치의 크기
sequence_len = 6      # 시퀀스 길이

# 입력 데이터 텐서를 랜덤하게 생성 (배치 크기 x 시퀀스 길이 x 입력 크기)
inputs = torch.randn(batch_size, sequence_len, input_size)

# 초기 은닉 상태(h_0)를 랜덤하게 생성
# 양방향인 경우, num_layers * 2 만큼의 은닉 상태를 설정함
h_0 = torch.rand(num_layers * (int(bidirectional) + 1), batch_size, ouput_size)

# 모델에 입력과 초기 은닉 상태를 전달하여 출력과 마지막 은닉 상태를 얻음
outputs, hidden = model(inputs, h_0)

# outputs는 모든 시퀀스 위치에서의 출력 값을 담고 있음
# 출력 크기는 (배치 크기, 시퀀스 길이, 은닉 상태 크기 * 양방향)임
print(outputs.shape)

# hidden은 각 계층의 마지막 은닉 상태를 담고 있음
# hidden 크기는 (계층 수 * 양방향, 배치 크기, 은닉 상태 크기)임
print(hidden.shape)

## 6.19 양방향 다층 장단기 메모리

In [ ]:
import torch
from torch import nn

# 모델의 입력 크기, 출력 크기, LSTM 층 개수, 양방향 및 프로젝션 크기를 정의함
input_size = 128    # 입력 벡터의 차원 크기
ouput_size = 256    # LSTM의 은닉 상태(hidden state) 크기
num_layers = 3      # LSTM 계층 수
bidirectional = True  # 양방향 LSTM 설정 여부
proj_size = 64      # 프로젝션(projection) 벡터 크기 (프로젝션을 통해 은닉 상태 크기를 줄임)

# LSTM 모델을 초기화함
model = nn.LSTM(
    input_size=input_size,       # 입력 벡터의 차원 크기 설정
    hidden_size=ouput_size,      # LSTM의 은닉 상태 크기 설정
    num_layers=num_layers,       # LSTM 계층 수 설정
    batch_first=True,            # 입력의 첫 번째 차원이 배치(batch) 크기임을 설정
    bidirectional=bidirectional, # 양방향 LSTM 설정
    proj_size=proj_size          # 프로젝션 크기 설정 (은닉 상태를 proj_size로 축소)
)

# 입력 데이터 설정
batch_size = 4       # 배치 크기
sequence_len = 6     # 시퀀스 길이

# 입력 데이터 텐서를 랜덤하게 생성 (배치 크기 x 시퀀스 길이 x 입력 크기)
inputs = torch.randn(batch_size, sequence_len, input_size)

# 초기 은닉 상태(h_0) 및 셀 상태(c_0)를 랜덤하게 생성
# 양방향 LSTM인 경우 num_layers * 2의 크기로 설정
# h_0는 프로젝션 크기만큼 축소된 크기 사용, 그렇지 않으면 은닉 상태 크기 사용
h_0 = torch.rand(
    num_layers * (int(bidirectional) + 1),  # 계층 수와 양방향 여부에 따른 크기
    batch_size,
    proj_size if proj_size > 0 else ouput_size  # 프로젝션 적용 시 크기, 그렇지 않으면 은닉 상태 크기
)

# 초기 셀 상태(c_0)를 은닉 상태 크기로 생성
c_0 = torch.rand(num_layers * (int(bidirectional) + 1), batch_size, ouput_size)

# 모델에 입력과 초기 은닉 상태 및 셀 상태를 전달하여 출력과 마지막 은닉 및 셀 상태를 얻음
outputs, (h_n, c_n) = model(inputs, (h_0, c_0))

# outputs는 모든 시퀀스 위치에서의 출력 값을 담고 있음
# 출력 크기는 (배치 크기, 시퀀스 길이, 프로젝션 크기 * 양방향)임
print(outputs.shape)

# h_n는 각 계층의 마지막 프로젝션된 은닉 상태를 담고 있음
# h_n 크기는 (계층 수 * 양방향, 배치 크기, 프로젝션 크기)임
print(h_n.shape)

# c_n는 각 계층의 마지막 셀 상태를 담고 있음
# c_n 크기는 (계층 수 * 양방향, 배치 크기, 은닉 상태 크기)임
print(c_n.shape)

## 6.20~6.27 문장 분류 모델 실습

In [ ]:
import torch
from torch import nn


# 문장 분류 모델 클래스 정의
class SentenceClassifier(nn.Module):
    def __init__(
        self,
        n_vocab,       # 어휘 사전 크기
        hidden_dim,    # 은닉 상태 크기
        embedding_dim, # 임베딩 크기
        n_layers,      # RNN 또는 LSTM 계층 수
        dropout=0.5,   # 드롭아웃 확률
        bidirectional=True, # 양방향 설정
        model_type="lstm"   # 모델 타입 ("rnn" 또는 "lstm")
    ):
        super().__init__()

        # 단어 임베딩 계층을 정의함
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0  # 패딩 토큰에 대해 0번 인덱스를 사용함
        )

        # RNN 또는 LSTM 계층을 정의함
        if model_type == "rnn":
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        # 양방향 여부에 따라 분류기 계층의 입력 차원 설정
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)

        # 드롭아웃 계층을 정의함
        self.dropout = nn.Dropout(dropout)

    # 순전파 계산을 정의함
    def forward(self, inputs):
        embeddings = self.embedding(inputs)  # 임베딩을 통해 입력을 임베딩 벡터로 변환
        output, _ = self.model(embeddings)   # RNN 또는 LSTM 계층 통과
        last_output = output[:, -1, :]       # 마지막 타임스텝의 출력만 선택
        last_output = self.dropout(last_output)  # 드롭아웃 적용
        logits = self.classifier(last_output)  # 분류기 통과하여 로짓 계산
        return logits

In [ ]:
import pandas as pd
from Korpora import Korpora

# NSMC 코퍼스를 불러와서 훈련 및 테스트 데이터로 나눔
corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)
train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 어휘 사전 생성 함수 정의
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)  # 각 문서의 토큰 카운트를 누적함
    vocab = special_tokens     # 특수 토큰 추가
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

# 토큰화 수행
tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

# 어휘 사전 구축 및 단어-인덱스 매핑 생성
vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

In [ ]:
import numpy as np

# 패딩 처리를 위한 함수 정의
def pad_sequences(sequences, max_length, pad_value):
    result = []
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

# 토큰을 인덱스로 변환하고 패딩 처리
unk_id = token_to_id["<unk>"]
train_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 텐서 데이터셋 및 데이터로더 설정
train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)
train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
from torch import optim

# 모델 학습 설정
n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)


In [ ]:
# 모델 학습 함수 정의
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


# 모델 테스트 함수 정의
def test(model, datasets, criterion, device):
    model.eval()
    losses = []
    corrects = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) > 0.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")

In [ ]:
# 모델 학습 및 테스트 실행
epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

In [ ]:
# 단어 임베딩 저장 및 특정 단어 확인
token_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word] = emb

token = vocab[1000]
print(token, token_to_embedding[token])

## 6.28~6.30 Word2Vec 모델을 활용한 문장 분류 모델 실습

In [ ]:
import torch
from torch import nn


# 문장 분류 모델 클래스 정의
class SentenceClassifier(nn.Module):
    def __init__(
        self,
        n_vocab,              # 어휘 사전 크기
        hidden_dim,           # 은닉 상태 크기
        embedding_dim,        # 임베딩 크기
        n_layers,             # RNN 또는 LSTM 계층 수
        dropout=0.5,          # 드롭아웃 확률
        bidirectional=True,   # 양방향 설정
        model_type="lstm",    # 모델 타입 ("rnn" 또는 "lstm")
        pretrained_embedding=None # 사전 학습된 임베딩
    ):
        super().__init__()

        # 사전 학습된 임베딩을 사용할 경우, 해당 임베딩을 적용
        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32)
            )
        # 사전 학습된 임베딩이 없을 경우, 새로운 임베딩 층 생성
        else:
            self.embedding = nn.Embedding(
                num_embeddings=n_vocab,
                embedding_dim=embedding_dim,
                padding_idx=0  # 패딩 토큰의 인덱스
            )

        # RNN 또는 LSTM 계층을 설정
        if model_type == "rnn":
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        # 양방향 여부에 따라 분류기 계층의 입력 크기 설정
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    # 순전파 계산 정의
    def forward(self, inputs):
        embeddings = self.embedding(inputs)   # 임베딩을 통해 입력 변환
        output, _ = self.model(embeddings)    # RNN 또는 LSTM 계층 통과
        last_output = output[:, -1, :]        # 마지막 타임스텝의 출력만 선택
        last_output = self.dropout(last_output)  # 드롭아웃 적용
        logits = self.classifier(last_output)    # 로짓 계산
        return logits

In [ ]:
import pandas as pd
from Korpora import Korpora

# NSMC 코퍼스를 불러와서 훈련 및 테스트 데이터로 나눔
corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)
train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 어휘 사전 생성 함수 정의
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)  # 각 문서의 토큰 카운트를 누적함
    vocab = special_tokens     # 특수 토큰 추가
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

# 토큰화 수행
tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

# 어휘 사전 구축 및 단어-인덱스 매핑 생성
vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

In [ ]:
import numpy as np

# 패딩 처리를 위한 함수 정의
def pad_sequences(sequences, max_length, pad_value):
    result = []
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

# 토큰을 인덱스로 변환하고 패딩 처리
unk_id = token_to_id["<unk>"]
train_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# 텐서 데이터셋 및 데이터로더 설정
train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)
train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
from gensim.models import Word2Vec

# 사전 학습된 Word2Vec 모델을 불러와서 임베딩 행렬 초기화
word2vec = Word2Vec.load("../models/word2vec.model")
init_embeddings = np.zeros((n_vocab, embedding_dim))

# 각 단어에 해당하는 임베딩을 초기화 행렬에 채움
for index, token in id_to_token.items():
    if token not in ["<pad>", "<unk>"]:
        init_embeddings[index] = word2vec.wv[token]

# 사전 학습된 임베딩으로부터 임베딩 층을 생성
embedding_layer = nn.Embedding.from_pretrained(
    torch.tensor(init_embeddings, dtype=torch.float32)
)

In [ ]:
# 모델 학습 함수 정의
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


# 모델 테스트 함수 정의
def test(model, datasets, criterion, device):
    model.eval()
    losses = []
    corrects = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) > 0.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")

In [ ]:
from torch import optim

# 모델 및 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim,
    n_layers=n_layers, pretrained_embedding=init_embeddings
).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

# 모델 학습 및 테스트 실행
epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

## 6.31 합성곱 모델

In [ ]:
import torch
from torch import nn

# CNN 클래스 정의
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 첫 번째 합성곱 계층 정의
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,     # 입력 채널 수 (RGB 이미지라서 3)
                out_channels=16,   # 출력 채널 수 (필터 수)
                kernel_size=3,     # 필터 크기
                stride=2,          # 필터 이동 간격
                padding=1          # 패딩 크기
            ),
            nn.ReLU(),            # 활성화 함수
            nn.MaxPool2d(kernel_size=2, stride=2),  # 2x2 최대 풀링
        )

        # 두 번째 합성곱 계층 정의
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,    # 이전 계층의 출력 채널 수가 입력 채널로 사용됨
                out_channels=32,   # 출력 채널 수
                kernel_size=3,     # 필터 크기
                stride=1,          # 필터 이동 간격
                padding=1          # 패딩 크기
            ),
            nn.ReLU(),            # 활성화 함수
            nn.MaxPool2d(kernel_size=2, stride=2),  # 2x2 최대 풀링
        )

        # 전결합 계층 (FC) 정의
        self.fc = nn.Linear(32 * 32 * 32, 10)  # 출력 뉴런 수는 10 (예: 클래스 10개)

    # 순전파 함수 정의
    def forward(self, x):
        x = self.conv1(x)         # 첫 번째 합성곱 계층 통과
        x = self.conv2(x)         # 두 번째 합성곱 계층 통과
        x = torch.flatten(x)      # 출력을 평탄화하여 FC 계층에 전달
        x = self.fc(x)            # 전결합 계층 통과
        return x

## 6.32~6.33 합성곱 신경망 분류 모델 실습

In [ ]:
import torch
from torch import nn

# 문장 분류 모델 클래스 정의
class SentenceClassifier(nn.Module):
    def __init__(self, pretrained_embedding, filter_sizes, max_length, dropout=0.5):
        super().__init__()

        # 사전 학습된 임베딩을 사용하여 임베딩 층 초기화
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(pretrained_embedding, dtype=torch.float32)
        )
        embedding_dim = self.embedding.weight.shape[1]  # 임베딩 차원 수

        # CNN 필터 생성
        conv = []
        for size in filter_sizes:
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=embedding_dim,  # 입력 채널 수 (임베딩 차원)
                        out_channels=1,            # 출력 채널 수
                        kernel_size=size           # 커널 크기
                    ),
                    nn.ReLU(),  # 활성화 함수
                    nn.MaxPool1d(kernel_size=max_length - size - 1),  # 맥스 풀링 적용
                )
            )
        self.conv_filters = nn.ModuleList(conv)  # CNN 필터 리스트로 저장

        # CNN 출력 크기에 따라 분류기 계층을 정의
        output_size = len(filter_sizes)
        self.pre_classifier = nn.Linear(output_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(output_size, 1)  # 최종 이진 분류기

    def forward(self, inputs):
        # 입력을 임베딩으로 변환하고 차원 변경 (배치, 채널, 길이 순서로 맞춤)
        embeddings = self.embedding(inputs)
        embeddings = embeddings.permute(0, 2, 1)

        # 각 CNN 필터를 적용하여 출력 리스트 생성
        conv_outputs = [conv(embeddings) for conv in self.conv_filters]
        # CNN 출력 텐서들을 결합
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)

        # 분류기 계층 통과 후 드롭아웃 및 최종 로짓 계산
        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        return logits

In [ ]:
import pandas as pd
from Korpora import Korpora

# NSMC 코퍼스를 불러와서 훈련 및 테스트 데이터로 나눔
corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)
train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 어휘 사전 생성 함수 정의
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)  # 각 문서의 토큰 카운트를 누적함
    vocab = special_tokens     # 특수 토큰 추가
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

# 토큰화 수행
tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

# 어휘 사전 구축 및 단어-인덱스 매핑 생성
vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

In [ ]:
import numpy as np

# 패딩 처리를 위한 함수 정의
def pad_sequences(sequences, max_length, pad_value):
    result = []
    for sequence in sequences:
        sequence = sequence[:max_length]  # 최대 길이에 맞게 자름
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length  # 패딩 값으로 채움
        result.append(padded_sequence)
    return np.asarray(result)

# 토큰을 인덱스로 변환하고 패딩 처리
unk_id = token_to_id["<unk>"]
train_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 텐서 데이터셋 및 데이터로더 설정
train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)
train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
from gensim.models import Word2Vec

# 사전 학습된 Word2Vec 모델을 불러와서 임베딩 행렬 초기화
n_vocab = len(token_to_id)
embedding_dim = 128
word2vec = Word2Vec.load("../models/word2vec.model")
init_embeddings = np.zeros((n_vocab, embedding_dim))

# 각 단어에 해당하는 임베딩을 초기화 행렬에 채움
for index, token in id_to_token.items():
    if token not in ["<pad>", "<unk>"]:
        init_embeddings[index] = word2vec.wv[token]

In [ ]:
from torch import optim

# 모델 및 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
filter_sizes = [3, 3, 4, 4, 5, 5]  # CNN 필터 크기 설정
classifier = SentenceClassifier(
    pretrained_embedding=init_embeddings,
    filter_sizes=filter_sizes,
    max_length=max_length
).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [ ]:
# 모델 학습 함수 정의
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


# 모델 테스트 함수 정의
def test(model, datasets, criterion, device):
    model.eval()
    losses = []
    corrects = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) > 0.5  # 예측값이 0.5보다 큰 경우 1로 분류
        corrects.extend(torch.eq(yhat, labels).cpu().tolist())

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")

In [ ]:
# 모델 학습 및 테스트 실행
epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)